# Peter Geiss HW 3B solution
Requires Pandas, numpy, and needs `train.csv` to be downloaded.

In [ ]:
import pandas as pd
import numpy as np

# You could do better than fillna(0) but for demonstration purposes I'll leave it like this.
df = pd.read_csv('train.csv').fillna(0) 
df.head(3)

In [ ]:
# First, let's make a consensus row for each day using average value.
group_by = df.groupby(['stock_id', 'date_id'], sort=False).mean()
group_by.head(3)

In [ ]:
#arr = group_by.reset_index()[['stock_id', 'reference_price', 'wap']].to_numpy() # smaller DF
arr = group_by.reset_index()[['stock_id', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap']].to_numpy() # bigger DF

print(arr.shape, arr[0])

We want to take distance of each (stock_id_a, stock_id_b, date_id) unique triple. My method creates a list of numpy arrays (e.g. `stocks[0]` represents all dates for stock_id 0) and each row in the array is one date_id (e.g. `stocks[0][0]` is the row for stock_id 0 on date_id 0).

Can you do better than this? Certainly. But the effort isn't worth the reward since the dataset isn't prohibitively large.

In [ ]:
###### Step 1: enumerate
# First, generate 200 numpy arrays, one for each stock_id. We'll do this with numpy masking.
# The mask I've chosen is `arr[:, 0] == i` which means "find rows whose first value is i". Each `i` is one stock_id.
#   This implies that `stocks[i]` gives stock_id i and `stocks[i][j]` gives date_id j for stock_id i.
# Additionally, there are some stock ids with missing values on some dates. I just resized to fill them in with zeroes.
#   In the real assignment, those can be handled better than I did here since doing this will offset the dates.

#stocks = [np.resize(arr[arr[:, 0] == i], (481, 3)) for i in range(0, 200)] # For smaller df
stocks = [np.resize(arr[arr[:, 0] == i], (481, 12)) for i in range(0, 200)] # For bigger df
print('Shape of stock_id 0 in `stocks`: ', stocks[0].shape)


###### Step 2: compute
# Now we need to use our list of numpy arrays to compute our distance metric for each stock on each day.
# I've included two simple distance functions, abs(b-a) and L2 norm AKA Euclidean distance.
# For illustrative purposes I've also generated a tuple that outputs the two stock_ids.

# The method of choice here is slicing. stocks[i][:,1:] means for the ith numpy array, for every row, take the 2nd+ column.
#   Naturally, we don't want the stock_id to be part of our distance calculation!
# Subtracting two equivalently sized numpy arrays performs element-wise subtraction using broadcasting.
# Performing L2 norm over axis 1 will broadcast a norm over each row instead of computing a single norm for all rows.
#   'np.abs' doesn't have that issue since it just applies element-wise abs.

#abs_dist = [(stocks[i][0][0], stocks[j][0][0], np.abs(stocks[i][:,1:]-stocks[j][:,1:])) for i in range(0, 200) for j in range(i+1,200)] # abs
L2_Norm = [(stocks[i][0][0], stocks[j][0][0], np.linalg.norm(stocks[i][:,1:]-stocks[j][:,1:], axis=1)) for i in range(0, 200) for j in range(i+1,200)] # L2 norm

#print([abs_dist[0][2].shape, abs_dist(L2_Norm), abs_dist[0]])
print('Size of outer list:', len(L2_Norm))
print('Shape of stock_id_a 0 compared with stock_id_b 1:', L2_Norm[0][2].shape)
print('Contents of stock_id_a 0 compared with stock_id_b 1:', L2_Norm[0])

For fun, I also attempted to compare every `(stock_id_a, stock_id_b, date_id_a, date_id_b)` tuple as well (around 900 million rows for each stock_id). It ran okay (around 4 seconds per stock_id so about 4 minutes to compute everything) until my machine ran out of memory. `top` was showing around 100 G of memory used when everything died. The method is nearly identical, but it needed some creativity with slicing on `np.abs`/`np.linalg.norm` step, something like `ls[i][:,None,:]-ls[j][:,:,None]`. 

It might have been doable if I stopped after each stock_id to remove distance values below a certain threshold but that leads to even more numpy padding problems since now you have jagged arrays.